In [13]:
import pandas as pd
import geopandas as gpd #allows for world map overlay
from shapely.geometry import Polygon

import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.colors as mcolors

import tkinter as tk
from tkinter import simpledialog, messagebox

import fsspec
from datetime import datetime
from shapely.geometry import box
import csv

In [15]:
#csv.reader("coordinates_dates_2022.csv")
df = pd.read_csv('coordinates_dates_2022.csv')
num_lines = df.shape[0]

# Print the number of lines in the CSV file
print("Number of lines in the CSV file: ", num_lines)

Number of lines in the CSV file:  194016


In [ ]:
# -------------------------------------------------------------------
# World Map Overlay
# -------------------------------------------------------------------
# Function to plot world map outline

url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/110m_cultural.zip"

def plot_world_map(ax):    
    with fsspec.open(f"simplecache::{url}") as file:
        world = gpd.read_file(file)
    #world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    world.plot(ax=ax, color='none', edgecolor='black', linewidth=1)

In [ ]:
%%time
df = pd.read_csv('coordinates_dates_2022.csv')
df['geometry'] = df.apply(lambda row: Polygon([
    (row['coordinate1_lon'], row['coordinate1_lat']),
    (row['coordinate2_lon'], row['coordinate2_lat']),
    (row['coordinate3_lon'], row['coordinate3_lat']),
    (row['coordinate4_lon'], row['coordinate4_lat']),
    (row['coordinate5_lon'], row['coordinate5_lat']),
]), axis=1)


In [ ]:
%%time
# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [ ]:
%%time
minx, miny, maxx, maxy = gdf.total_bounds
width = maxx - minx
height = maxy - miny

# Define the number of divisions in the grid
divisions = 150

In [ ]:
%%time
# Initialize an empty list to hold the data dictionaries
full_year_data_list = []

# Populate the list with counts
for i in range(divisions):
    for j in range(divisions):
        # Define the corners of the grid cell
        cell_minx = minx + i * (width / divisions)
        cell_miny = miny + j * (height / divisions)
        cell_maxx = minx + (i + 1) * (width / divisions)
        cell_maxy = miny + (j + 1) * (height / divisions)
        cell = Polygon([(cell_minx, cell_miny), (cell_maxx, cell_miny), (cell_maxx, cell_maxy), (cell_minx, cell_maxy)])
        count = gdf[gdf.geometry.intersects(cell)].shape[0]
        full_year_data_list.append({'geometry': cell, 'count': count})

In [ ]:
%%time
# Convert the list of dictionaries to a GeoDataFrame
heatmap_gdf = gpd.GeoDataFrame(full_year_data_list)


In [ ]:
heatmap_gdf.to_csv('polygon_only_2022.csv', index=False)